### Running session-level frequency-based modelling from this notebook

In [ ]:
%%time

experiment_id = "1_frequency_tagging"
time_window_id = "truncate-1-219"
design_matrix_id = "00_experiment-min"
mri_id = "3T"
fla_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level"
out_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/session_level"
sub_id = '002'
ses_id = '01'
n_permutations = 10
search_frequencies = [.125, .2]

!python3 /opt/app/scripts/hla.py session-level-entrain-gt-control \
    {experiment_id} \
    {time_window_id} \
    {design_matrix_id} \
    {mri_id} \
    {fla_dir} \
    {out_dir} \
    {sub_id} \
    {ses_id} \
    {n_permutations} \
    {' '.join([str(i) for i in search_frequencies])}

### Running session-level frequency-based modelling on ComputeCanada

In [ ]:
from pathlib import Path

def check_list_for_tasks(_list):
    entrain = False
    control = False

    for e in _list:
        if e.startswith('task-control'):
            entrain = True
        elif e.startswith('task-entrain'):
            control = True

    if entrain and control:
        return True
    else:
        return False

scripts_dir = "/data/scripts/03b_session_level"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

CONTAINER_DIR= '/project/def-mmur/gngo4/containers'

experiment_search_frequencies = {
    "7T": [.2, .5],
    "3T": [.125, .2],
}

n_permutations = 200
experiment_id = "1_frequency_tagging"
mri_ids = ["3T","7T"]

for mri_id in mri_ids:

    fla_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level"
    out_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/session_level"
    
    time_window_ids = !ls {fla_dir}
    for time_window_id in time_window_ids:
        
        design_matrix_ids = !ls {fla_dir}/{time_window_id}
        for design_matrix_id in design_matrix_ids:

            sub_ids = !ls {fla_dir}/{time_window_id}/{design_matrix_id}
            for sub_id in sub_ids:

                if not sub_id.startswith('sub-'):
                    continue

                ses_ids = !ls {fla_dir}/{time_window_id}/{design_matrix_id}/{sub_id}
                for ses_id in ses_ids:
                    task_ids = !ls {fla_dir}/{time_window_id}/{design_matrix_id}/{sub_id}/{ses_id}

                    if not check_list_for_tasks(task_ids):
                        continue

                    print(
                        experiment_id, 
                        mri_id, 
                        time_window_id, 
                        design_matrix_id, 
                        sub_id.split('-')[-1], 
                        ses_id.split('-')[-1], 
                        n_permutations, 
                        experiment_search_frequencies[mri_id]
                    )

                    txt = f"""#!/bin/bash
module load apptainer/1.1

singularity run \\
    --bind /project/def-mmur/gngo4/data/fastfmri:/data \\
    --bind /scratch/gngo4:/scratch \\
    --bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
    {CONTAINER_DIR}/neuroimaging-notebook-v2.simg \\
    python3 /opt/app/scripts/hla.py session-level-entrain-gt-control \\
    {experiment_id} \\
    {time_window_id} \\
    {design_matrix_id} \\
    {mri_id} \\
    {fla_dir} \\
    {out_dir} \\
    {sub_id.split('-')[-1]} \\
    {ses_id.split('-')[-1]} \\
    {n_permutations} \\
    {' '.join([str(i) for i in experiment_search_frequencies[mri_id]])}
                    """
                    cmd_path = f"{scripts_dir}/{experiment_id}_{mri_id}.{time_window_id}.{design_matrix_id}.{mri_id}.{sub_id}.{ses_id}.sh"
                    with open(cmd_path, 'w') as f:
                        f.write(txt)

In [ ]:
#!sbatch --time=03:00:00 --mem=8GB --account=def-mmur {cmd_path}